In [ ]:
include("ABLBreIF/BLBreIF.jl")
import .BLBreIF
include("SCNMF/SCNMF.jl")
import .SCNMF
using Images

In [ ]:
ul = "ORL_Faces/s1/1.pgm"
ig = load(ul)

a, b = size(ig)
R = 5

B = Matrix{Float64}(undef, a*b, 400)
for i = 1 : 40
    for j = 1 : 10
        local url = "ORL_Faces/s"*"$i/"*"$j"*".pgm"
        local img = load(url)
        local q = float64.(channelview(img))
        q = q[:]
        global B[:, (i-1)*10 + j] = q
    end
end
A = SCNMF.normalize!(B)
X, Y = SCNMF.randinit(A, R^2, 0.8)

In [ ]:
ρ₀ = 0.8 # step size parameter
rtime = 60 # runtime

In [ ]:
X₀ = copy(X);
Y₀ = copy(Y);
r₀ = SCNMF.solve!(SCNMF.BPG{Float64}(obj=:cons,
        constraint=0.95,
        runtime=rtime,
        ρ=ρ₀), A, X₀, Y₀)

X₁ = copy(X);
Y₁ = copy(Y);
r₁ = SCNMF.solve!(SCNMF.BBPG{Float64}(obj=:cons,
        constraint=0.95,
        runtime=rtime,
        ρ=ρ₀), A, X₁, Y₁)


X₂ = copy(X);
Y₂ = copy(Y);
r₂ = BLBreIF.solve!(BLBreIF.ALBreI{Float64}(runtime=rtime,
        verbose=false,
        ρ=ρ₀,
        μ₁=0.001,
        μ₂=0.001), A, X₂, Y₂)

X₃ = copy(X);
Y₃ = copy(Y);
r₃ = BLBreIF.solve!(BLBreIF.ABLBreI{Float64}(runtime=rtime,
        verbose=false,
        ρ=ρ₀,
        μ₁=0.001,
        μ₂=0.001), A, X₃, Y₃)

In [ ]:
stop₀ = r₀.niters
pic₀ = r₀.objvalue
rt₀ = pic₀[1:stop₀, 1]
obj₀ = pic₀[1:stop₀, 2]

stop₁ = r₁.niters
pic₁ = r₁.objvalue
rt₁ = pic₁[1:stop₁, 1]
obj₁ = pic₁[1:stop₁, 2]

stop₂ = r₂.niters
pic₂ = r₂.objvalue
rt₂ = pic₂[1:stop₂, 1]
obj₂ = pic₂[1:stop₂, 2]

stop₃ = r₃.niters
pic₃ = r₃.objvalue
rt₃ = pic₃[1:stop₃, 1]
obj₃ = pic₃[1:stop₃, 2]

In [ ]:
using CairoMakie
using Makie
using LaTeXStrings
using GLMakie
using Colors
using AlgebraOfGraphics
CairoMakie.activate!()
function speed()
    lines(rt₀, sqrt.(2*obj₀); color="#4063D8", label="BPG", linewidth=3, linestyle=:dashdot,
            figure=(; figure_padding=50, resolution=(1200, 800), font="sans",
                    backgroundcolor=:white, fontsize=32),
            axis=(; xlabel="Time(sec)", ylabel=L"\Vert A-XY\Vert_F",
                    #yscale=log10,
                    xgridstyle=:dash, ygridstyle=:dash))
    lines!(rt₁, sqrt.(2*obj₁); color="#389826", linewidth=3, linestyle=:dashdot,
            label="BBPG")
    lines!(rt₂, sqrt.(2*obj₂); color="#FFC633", linewidth=2, linestyle=:solid,
            label="ALBreI")
    lines!(rt₃, sqrt.(2*obj₃); color="#CB3C33", linewidth=2, linestyle=:solid,
            label="ABLBreI")
    limits!(0, 120, 0.8, 1.0)
    axislegend("Algorithms"; merge=true)
    current_figure()
end

with_theme(speed, theme=theme_dark())
save("plot/real_data.png", speed())